In [1]:
from pyscf import gto, dft, scf

In [17]:
from PySCFAdapter import PySCFAdapter
from DQCAdapter import DQCAdapter

In [3]:
water = """
O        0.000000000      0.000000000      0.000000000;
H        0.000000000      1.434938863      1.126357947;
H        0.000000000     -1.434938863      1.126357947
"""

h2_system = "H 0 0 0; H 1.4 0 0"

n2_system = "N 0 0 0; N 2.07 0 0"

ammonia = """
N  0.000  0.000  0.000; 
H  0.000 -1.772 -0.721; 
H  1.535  0.886 -0.721; 
H -1.535  0.886 -0.721
"""

In [4]:
from math import pi

factor = -3.0/4.0 * (3.0 / pi)**(1.0/3.0)
power = 4.0/3.0

In [13]:
import torch
import dqc
import dqc.xc
import dqc.utils


class MyLDAX(dqc.xc.CustomXC):
    def __init__(self, a_par, p_par):
        super().__init__()
        self.a_par = a_par
        self.p_par = p_par
        self.number_of_parameters = 2

    @property
    def family(self):
        # 1 for LDA, 2 for GGA, 4 for MGGA
        return 1

    def get_edensityxc(self, densinfo):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc(densinfo.u * 2) + self.get_edensityxc(densinfo.d * 2))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            return self.a_par * rho ** self.p_par
        
    def get_edensityxc_derivative(self, densinfo, number_of_parameter):
        # densinfo has up and down components
        if isinstance(densinfo, dqc.utils.SpinParam):
            # spin-scaling of the exchange energy
            return 0.5 * (self.get_edensityxc_derivative(densinfo.u * 2, number_of_parameter) 
                          + self.get_edensityxc_derivative(densinfo.d * 2, number_of_parameter))
        else:
            rho = densinfo.value.abs() + 1e-15  # safeguarding from nan
            if number_of_parameter == 0: # parameter a
                return rho ** self.p_par
            elif number_of_parameter == 1: # parameter p
                return self.a_par * torch.log(rho) * rho ** self.p_par
            
a_par = torch.nn.Parameter(torch.tensor(factor, dtype=torch.double))
p_par = torch.nn.Parameter(torch.tensor(power, dtype=torch.double))
myxc = MyLDAX(a_par, p_par)

In [18]:
mol_dqc = dqc.Mol(moldesc=h2_system, basis="6-31G")
qc_dqc = dqc.KS(mol_dqc, xc=myxc).run()
ene = qc_dqc.energy()
dqc_adapter = DQCAdapter(qc_dqc)

density info torch.Size([33420])
started the four term integral computation for torch.Size([33420, 4])...
g_p: tensor([0.7484, 1.4715], dtype=torch.float64)
g_x^t norm: 1.386610552568249
lambda norm: 1.3128781020836782
f_p norm: 1.43435492572186
lambda_t * f_p: tensor([ 1.5958e-16, -4.1825e-17], dtype=torch.float64)
f_x^-t norm: 9.160860424774633
f_x^-1 * f_p norm: 0.8323438554981616
g_x^t * f_x^-1 * f_p: tensor([ 1.6404e-16, -4.1557e-17], dtype=torch.float64)


In [19]:
print("Modified overlap:\n", dqc_adapter.get_overlap_matrix())
print("Modified Fockian:\n", dqc_adapter.get_fockian())
print("Orbital energies", dqc_adapter.get_orbital_energies())
print("Modified orbital coefficients:\n", dqc_adapter.get_orbital_coefficients())
print("Orbital occupancies", dqc_adapter.get_orbital_occupancy())
print("Number of occupied orbitals:", dqc_adapter.get_number_of_occupied_orbitals())
print("Number of all orbitals:", dqc_adapter.get_number_of_all_orbitals())
print("Number of DFT parameters:", dqc_adapter.get_number_of_parameters())
print("Derivative of exc with respect to parameters:\n", dqc_adapter.get_derivative_of_exc_wrt_theta())
print("Derivative of vxc with respect to parameters:\n", dqc_adapter.get_derivative_of_vxc_wrt_theta())

Modified overlap:
 tensor([[ 1.0000e+00,  1.2075e-15,  1.1112e-16,  6.6457e-16],
        [ 1.0546e-15,  1.0000e+00, -1.9159e-16,  1.2880e-16],
        [ 5.1892e-16, -1.6251e-16,  1.0000e+00,  2.1072e-16],
        [ 9.1132e-16,  2.1353e-16,  2.5082e-16,  1.0000e+00]],
       dtype=torch.float64)
Modified Fockian:
 tensor([[ 6.8396e-01,  5.6922e-16, -5.0047e-01,  3.8433e-16],
        [ 6.7048e-16,  5.9425e-01,  8.4385e-17,  5.3440e-02],
        [-5.0047e-01,  2.4980e-16,  5.5621e-01, -1.1752e-16],
        [ 2.3624e-16,  5.3440e-02, -1.7168e-16, -3.2512e-01]],
       dtype=torch.float64)
Orbital energies tensor([-0.3282], dtype=torch.float64)
Modified orbital coefficients:
 tensor([[-7.0637e-16],
        [-5.7835e-02],
        [-3.2083e-16],
        [ 9.9833e-01]], dtype=torch.float64)
Orbital occupancies tensor([2.], dtype=torch.float64)
Number of occupied orbitals: 1
Number of all orbitals: 4
Number of DFT parameters: 2
Derivative of exc with respect to parameters:
 tensor([0.7484, 1.47

In [5]:
import numpy

def custom_eval_xc(a, p):
    def eval_xc(xc_code, rho, *args, **kwargs):
        exc = a * rho ** (p - 1.0)
        vrho = a * p * rho ** (p - 1.0)
        vxc = (vrho, None, None, None)
        fxc = a * p * (p - 1.0) * rho ** (p - 2.0)  # 2nd order functional derivative
        kxc = None  # 3rd order functional derivative
        return exc, vxc, fxc, kxc
    def eval_xc_wrt_a(xc_code, rho, *args, **kwargs):
        dexc_wrt_da = rho ** (p - 1.0)
        dvrho_wrt_da = p * rho ** (p - 1.0)
        dvxc_wrt_da = (dvrho_wrt_da, None, None, None)
        return dexc_wrt_da, dvxc_wrt_da, None, None
    def eval_xc_wrt_p(xc_code, rho, *args, **kwargs):
        dexc_wrt_dp = a * numpy.log(rho) * rho ** (p - 1.0)
        dvrho_wrt_dp = a * p * numpy.log(rho) * rho ** (p - 1.0)
        dvxc_wrt_dp = (dvrho_wrt_dp, None, None, None)
        return dexc_wrt_dp, dvxc_wrt_dp, None, None
    eval_xc.number_of_parameters = 2
    eval_xc.wrt = [eval_xc_wrt_a, eval_xc_wrt_p]
    return eval_xc

In [6]:
mol = gto.M(atom = h2_system, basis="6-31G", unit = 'Bohr')
calc = dft.RKS(mol)
calc = calc.define_xc_(custom_eval_xc(factor, power), 'LDA')
ene = calc.kernel()

converged SCF energy = -1.03861778567388


In [8]:
qc = calc
dm_in_nonorthogonal_basis = qc.make_rdm1()
coords_of_grid = qc.grids.coords
weights_of_grid = qc.grids.weights
ao_value_on_grid = dft.numint.eval_ao(qc.mol, coords_of_grid, deriv=0)
rho_on_grid = dft.numint.eval_rho(qc.mol, ao_value_on_grid, dm_in_nonorthogonal_basis, xctype='LDA')
qc._numint.eval_xc('LDA', rho_on_grid)[1][0].shape
ao_value_on_grid.shape

(17308, 4)

In [9]:
adapter = PySCFAdapter(calc)

In [10]:
print("Initial overlap:\n", calc.get_ovlp())
print("Modified overlap:\n", adapter.get_overlap_matrix())
print("Initial Fockian:\n", calc.get_fock())
print("Modified Fockian:\n", adapter.get_fockian())
print("Orbital energies", adapter.get_orbital_energies())
print("Initial orbital coefficients:\n", scf.hf.kernel(calc)[3])
print("Modified orbital coefficients:\n", adapter.get_orbital_coefficients())
print("Orbital occupancies", adapter.get_orbital_occupancy())
print("Number of occupied orbitals:", adapter.get_number_of_occupied_orbitals())
print("Number of all orbitals:", adapter.get_number_of_all_orbitals())
print("Number of DFT parameters:", adapter.get_number_of_parameters())
print("Derivative of exc with respect to parameters:\n", adapter.get_derivative_of_exc_wrt_theta())
print("Derivative of vxc with respect to parameters:\n", adapter.get_derivative_of_vxc_wrt_theta())

Initial overlap:
 [[1.         0.65829205 0.45453899 0.50876163]
 [0.65829205 1.         0.50876163 0.85380517]
 [0.45453899 0.50876163 1.         0.65829205]
 [0.50876163 0.85380517 0.65829205 1.        ]]
Modified overlap:
 [[ 1.00000000e+00  2.82300720e-16  1.19720313e-15  6.81910665e-16]
 [-4.64167708e-15  1.00000000e+00 -3.41826478e-15 -2.59359187e-15]
 [-4.56201820e-16 -8.07188108e-16  1.00000000e+00 -4.14697636e-16]
 [ 4.79416190e-15  4.69297585e-15  2.88264341e-15  1.00000000e+00]]
Initial Fockian:
 [[ 0.04765594 -0.30967894 -0.32883878 -0.29320283]
 [-0.30967894 -0.17082453 -0.29320283 -0.19135331]
 [-0.32883878 -0.29320283  0.04765594 -0.30967894]
 [-0.29320283 -0.19135331 -0.30967894 -0.17082453]]
Modified Fockian:
 [[ 0.51419684 -0.4504878  -0.3548006  -0.0116299 ]
 [-0.4504878   0.24045611 -0.0116299  -0.13072311]
 [-0.3548006  -0.0116299   0.51419684 -0.4504878 ]
 [-0.0116299  -0.13072311 -0.4504878   0.24045611]]
Orbital energies [-0.32821985  0.11555682  0.59734892  1.1

$FC=SCE$

$FS^{-\frac{1}{2}}S^{\frac{1}{2}}C=SCE$

$FS^{-\frac{1}{2}}S^{\frac{1}{2}}C=S^{\frac{1}{2}}S^{\frac{1}{2}}CE$

$\hat{C} = S^{\frac{1}{2}}C$

$FS^{-\frac{1}{2}}\hat{C}=S^{\frac{1}{2}}\hat{C}E$

$S^{-\frac{1}{2}}FS^{-\frac{1}{2}}\hat{C}=\hat{C}E$

$S^{-\frac{1}{2}}FS^{-\frac{1}{2}} = \hat{F}$

$\hat{F}\hat{C}=\hat{C}E$

In [11]:
import scipy
mol = gto.M(atom = h2_system, basis="6-31G", unit = 'Bohr')
calc = dft.RKS(mol)
ene = calc.kernel()
basis_set_orthogonalization_matrix = scipy.linalg.fractional_matrix_power(calc.get_ovlp(), -0.5)
basis_set_orthogonalization_matrix

converged SCF energy = -1.13266912566127


array([[ 1.29356261, -0.50155666, -0.19466961,  0.07534272],
       [-0.50155666,  1.99085479,  0.07534272, -1.03777583],
       [-0.19466961,  0.07534272,  1.29356261, -0.50155666],
       [ 0.07534272, -1.03777583, -0.50155666,  1.99085479]])